In [126]:
import altair

# Data Extraction

In [127]:
from dotenv import load_dotenv
import os
from Crypto.Hash import MD5
import requests
import time

In [128]:
class DataExtractor:

    __url:str = "http://gateway.marvel.com/v1/public/"
    __endpoints:str = ["characters", "comics", "creators", "events", "series", "stories"]
    __priv_key:str = None
    __pub_key:str = None

    def __init__(self):  
        """
        Extracts data from the Marvel API (developer.marvel.com).
        In order to create an object of this class, there must exist a .env file with the API key and private key, 
        necessary to access the API, under the names "PUB_KEY" and "PRIV_KEY".
        """

        load_dotenv()
        self.__priv_key = os.getenv("PRIV_KEY")
        self.__pub_key = os.getenv("PUB_KEY")

    def __format_request_url(self, endpoint:str) -> str:
        """
        Formats the URL to be used in a request, using the timestamp, public and private keys and the desired endpoint.

        Args:
            endpoint (str): The endpoint of the request URL.

        Returns:
            str: The formatted request URL.

        Raises:
            ValueError: If the provided endpoint is not valid.
        """

        # Verifies the endpoint exists in the API
        if endpoint not in self.__endpoints:
            raise ValueError(f"{endpoint} is not a valid endpoint.")

        timestamp = time.time()
        timestamp_str = f"ts={timestamp}"

        apikey_str = "apikey=" + self.__pub_key

        # Calculates the MD5 hash, needed for the URL
        md5_message = str(timestamp) + self.__priv_key + self.__pub_key
        hash_str = "hash=" + MD5.new(str.encode(md5_message)).hexdigest()

        request_url = self.__url + endpoint + "?" + timestamp_str + "&" + apikey_str + "&" + hash_str
        return request_url
    
    def __make_request(self, request_url:str) -> list[dict]:
        """
        Makes a request to the API.

        Args:
            request_url (str): The request URL.

        Returns:
            list[dict]: The data of the response in JSON, each item in the list is an entry on the database.

        Raises:
            requests.HTTPError: If the received status code is not 200.
        """

        # Makes the request
        response = requests.get(request_url)

        if response.status_code != 200:
            raise requests.HTTPError(f"Received status code {response.status_code} for {response.url}")
        
        # Filters metadata out
        return response.json()["data"]["results"]

    def extract(self) -> dict:
        """
        Extracts the data from all API endpoints.
        
        Returns:
            The API data, where the key-value pairs are the name of the resource and its 
            data in JSON format (python dictionary), if the request was successful, and None otherwise.
        """

        api_data = {}

        # For each endpoints, tries to extract the data and add it to the api_data dictionary
        for endpoint in self.__endpoints:
            try:
                request_url = self.__format_request_url(endpoint)
                endpoint_data = self.__make_request(request_url)
                api_data[endpoint] = endpoint_data

            except Exception as e:
                print(f"Error: {e}")

                # If the request was unsuccessful, adds the endpoint key to the dict with null data
                api_data[endpoint] = None 
        
        return api_data

In [129]:
de = DataExtractor()
data_json = de.extract()

# Data Transformation

In [130]:
import pandas as pd
import numpy as np

In [138]:
class DataTransformator:
    
    data_json:dict[str, dict] = None
    data_DFs:dict[str, pd.DataFrame] = None

    def __init__(self, data_json:dict[str, dict] = None, data_DFs:dict[str, pd.DataFrame] = None):
        """
        Converts the data returned by the Marvel API into Pandas' DataFrames, cleaning the data.

        Args:
            data_json: Data to be transformed, in JSON. Each key (str) is a resource from the API and the value (dict) is the information.
            data_DFs: Data to be transformed, in DataFrame. Each key (str) is a resource from the API and the value (pd.DataFrame) is the information.
        """
        self.data_DFs = data_DFs
        self.data_json = data_json

    # STATIC METHODS
    # These are used with pd.DataFrame.apply() in the other class methods

    @staticmethod
    def __convert_timestamp(timestamp:str) -> pd.Timestamp:
        """
        Converts a string in Pandas' Timestamp.
        
        Args:
            timestamp: The timestamp in string type.

        Returns:
            pd.Timestamp: The timestamp in pd.Timestamp format, or None if string was invalid.
        """

        if timestamp == None or timestamp[0] == "-":
            return None
        else:
            return pd.to_datetime(timestamp)
        
    @staticmethod
    def __get_resources_ids(original_dict:dict) -> list[int]:
        """
        Get resources' IDs from a collection of resources.

        Args:
            original_dict: Collection of resources returned by the API.
        
        Returns:
            list[int]: List of IDs of all the resources on the collection.
        """

        items = original_dict["items"]

        ids = []
        for item in items:
            ids.append(int(item["resourceURI"].split("/")[-1]))

        return ids
    
    @staticmethod
    def __get_resource_id(original_dict:dict) -> int:
        """
        Get resource's IDs from a resource.

        Args:
            original_dict: Resource returned by the API.
        
        Returns:
            int: ID of the resource.
        """

        if original_dict != None:
            return original_dict["resourceURI"].split("/")[-1]
        else:
            return None
    
    @staticmethod
    def __get_sale_date(dates:list[dict]) -> pd.Timestamp:
        """
        Selects the sale date from the list of dates.

        Args:
            dates: Dictionary of dates returned from the API.
        
        Returns:
            pd.Timestamp: Sale date.
        """

        for item in dates:
            if item["type"] == "onsaleDate":
                return pd.to_datetime(item["date"])
            
    @staticmethod
    def __get_foc_date(dates:list[dict]) -> pd.Timestamp:
        """
        Selects the FOC date from the list of dates.

        Args:
            dates: Dictionary of dates returned from the API.
        
        Returns:
            pd.Timestamp: FOC date.
        """

        for item in dates:
            if item["type"] == "focDate":
                return DataTransformator.__convert_timestamp(item["date"])
            
    @staticmethod
    def __get_print_price(prices:list[dict]) -> float:
        """
        Selects the print price from the list of prices.

        Args:
            prices: Dictionary of prices returned from the API.
        
        Returns:
            pd.Timestamp: Print price.
        """

        for price in prices:
            if price["type"] == "printPrice":
                return price["price"]
            
    @staticmethod
    def __get_digital_price(prices:list[dict]) -> float:
        """
        Selects the digital price from the list of prices.

        Args:
            prices: Dictionary of prices returned from the API.
        
        Returns:
            pd.Timestamp: Digital price.
        """

        for price in prices:
            if price["type"] == "digitalPurchasePrice":
                return price["price"]
            
    # PRIVATE CLASS METHODS
    # These are used by the public method "transform". They modify the object self.data_DFs, 
    # except for "__convert_to_raw_dataframes", that reads from self.data_json and stores in self.data_DFs.
    
    def __convert_to_raw_dataframes(self):
        """
        Converts the data returned by the API in JSON, stored in self.data_json, into DataFrame objects and stores into self.data_DFs.
        """

        self.data_DFs = {}

        for item in self.data_json:
            self.data_DFs[item] = pd.DataFrame(self.data_json[item])
    
    def __create_new_columns(self):
        """
        Separates columns of type list[dict] with multiple information into multiple columns, each with one information.
        """

        self.data_DFs["comics"]["saleDate"] = self.data_DFs["comics"]["dates"].apply(self.__get_sale_date)
        self.data_DFs["comics"]["focDate"] = self.data_DFs["comics"]["dates"].apply(self.__get_foc_date)

        self.data_DFs["comics"]["printPrice"] = self.data_DFs["comics"]["prices"].apply(self.__get_print_price)
        self.data_DFs["comics"]["digitalPrice"] = self.data_DFs["comics"]["prices"].apply(self.__get_digital_price)

    def __convert_datatypes(self):
        """
        Converts timestamp columns from string to pd.Timestamp.
        """

        self.data_DFs["characters"]["modified"] = self.data_DFs["events"]["modified"].apply(self.__convert_timestamp)
        self.data_DFs["comics"]["modified"] = self.data_DFs["events"]["modified"].apply(self.__convert_timestamp)
        self.data_DFs["creators"]["modified"] = self.data_DFs["events"]["modified"].apply(self.__convert_timestamp)
        self.data_DFs["series"]["modified"] = self.data_DFs["events"]["modified"].apply(self.__convert_timestamp)
        self.data_DFs["stories"]["modified"] = self.data_DFs["events"]["modified"].apply(self.__convert_timestamp)
        self.data_DFs["events"]["modified"] = self.data_DFs["events"]["modified"].apply(self.__convert_timestamp)

        self.data_DFs["events"]["start"] = self.data_DFs["events"]["start"].apply(self.__convert_timestamp)
        self.data_DFs["events"]["end"] = self.data_DFs["events"]["end"].apply(self.__convert_timestamp)

    def __identify_resources_by_id(self):
        """
        Operates on columns where each cell has type list[dict], storing lists of resources returned from the API. 
        Extracts only the ID of the resources, making the column's cells of type list[int].
        """

        self.data_DFs["stories"]["events"] = self.data_DFs["stories"]["events"].apply(self.__get_resources_ids)
        self.data_DFs["stories"]["creators"] = self.data_DFs["stories"]["creators"].apply(self.__get_resources_ids)
        self.data_DFs["stories"]["series"] = self.data_DFs["stories"]["series"].apply(self.__get_resources_ids)
        self.data_DFs["stories"]["comics"] = self.data_DFs["stories"]["comics"].apply(self.__get_resources_ids)
        self.data_DFs["stories"]["characters"] = self.data_DFs["stories"]["characters"].apply(self.__get_resources_ids)

        self.data_DFs["series"]["comics"] = self.data_DFs["series"]["comics"].apply(self.__get_resources_ids)
        self.data_DFs["series"]["events"] = self.data_DFs["series"]["events"].apply(self.__get_resources_ids)
        self.data_DFs["series"]["stories"] = self.data_DFs["series"]["stories"].apply(self.__get_resources_ids)
        self.data_DFs["series"]["creators"] = self.data_DFs["series"]["creators"].apply(self.__get_resources_ids)
        self.data_DFs["series"]["characters"] = self.data_DFs["series"]["characters"].apply(self.__get_resources_ids)
        self.data_DFs["series"]["next"] = self.data_DFs["series"]["next"].apply(self.__get_resource_id)
        self.data_DFs["series"]["previous"] = self.data_DFs["series"]["previous"].apply(self.__get_resource_id)

        self.data_DFs["events"]["comics"] = self.data_DFs["events"]["comics"].apply(self.__get_resources_ids)
        self.data_DFs["events"]["series"] = self.data_DFs["events"]["series"].apply(self.__get_resources_ids)
        self.data_DFs["events"]["stories"] = self.data_DFs["events"]["stories"].apply(self.__get_resources_ids)
        self.data_DFs["events"]["creators"] = self.data_DFs["events"]["creators"].apply(self.__get_resources_ids)
        self.data_DFs["events"]["characters"] = self.data_DFs["events"]["characters"].apply(self.__get_resources_ids)
        self.data_DFs["events"]["next"] = self.data_DFs["events"]["next"].apply(self.__get_resource_id)
        self.data_DFs["events"]["previous"] = self.data_DFs["events"]["previous"].apply(self.__get_resource_id)

        self.data_DFs["creators"]["comics"] = self.data_DFs["creators"]["comics"].apply(self.__get_resources_ids)
        self.data_DFs["creators"]["series"] = self.data_DFs["creators"]["series"].apply(self.__get_resources_ids)
        self.data_DFs["creators"]["stories"] = self.data_DFs["creators"]["stories"].apply(self.__get_resources_ids)
        self.data_DFs["creators"]["events"] = self.data_DFs["creators"]["events"].apply(self.__get_resources_ids)

        self.data_DFs["comics"]["series"] = self.data_DFs["comics"]["series"].apply(self.__get_resource_id)
        self.data_DFs["comics"]["stories"] = self.data_DFs["comics"]["stories"].apply(self.__get_resources_ids)
        self.data_DFs["comics"]["events"] = self.data_DFs["comics"]["events"].apply(self.__get_resources_ids)
        self.data_DFs["comics"]["creators"] = self.data_DFs["comics"]["creators"].apply(self.__get_resources_ids)
        self.data_DFs["comics"]["characters"] = self.data_DFs["comics"]["characters"].apply(self.__get_resources_ids)

        self.data_DFs["characters"]["comics"] = self.data_DFs["characters"]["comics"].apply(self.__get_resources_ids)
        self.data_DFs["characters"]["series"] = self.data_DFs["characters"]["series"].apply(self.__get_resources_ids)
        self.data_DFs["characters"]["stories"] = self.data_DFs["characters"]["stories"].apply(self.__get_resources_ids)
        self.data_DFs["characters"]["events"] = self.data_DFs["characters"]["events"].apply(self.__get_resources_ids)

    def __drop_irrelevant_columns(self):
        """
        Drops colums that were deemed irrelevant for the analysis in this notebook.
        """

        self.data_DFs["events"].drop(["description", "thumbnail", "resourceURI", "urls"], axis=1, inplace=True)
        self.data_DFs["stories"].drop(["description", "thumbnail", "resourceURI", "type", "originalIssue"], axis=1, inplace=True)
        self.data_DFs["series"].drop(["description", "thumbnail", "resourceURI", "urls"], axis=1, inplace=True)
        self.data_DFs["creators"].drop(["thumbnail", "resourceURI", "urls"], axis=1, inplace=True)
        self.data_DFs["characters"].drop(["description", "thumbnail", "resourceURI", "urls"], axis=1, inplace=True)
        self.data_DFs["comics"].drop(["digitalId", "variantDescription", "description", "isbn", "resourceURI", "urls", "upc", 
                                      "diamondCode", "ean", "issn", "format", "variants", "textObjects", "collections", 
                                      "collectedIssues", "thumbnail", "images", "dates", "prices"], axis=1, inplace=True)
    
    def __filter_list_columns(self):
        """
        Filter columns where each cell stores a list and checks for column where all the lists have 0 elements (then drops the column) 
        or all the lists have at most 1 element (then reduces the column from list to only the first element or None when empty).
        """

        # For each df in the database
        for topic, df in self.data_DFs.items():
            
            # Checks what columns stores a list in each cell
            is_list:list[np.bool] = (df.map(type) == list).all()

            for column in df.columns:
                if is_list[column]:

                    # Calculates the lenght of each cell's list
                    lenght_of_cells = df[column].apply(len)

                    # Count how many cells are empty lists
                    non_empty_cells_count = len(df[lenght_of_cells > 0].index)

                    # Count how many calls have at most one element
                    greater_than_one_cells_count = len(df[lenght_of_cells > 1].index)

                    # If all cells are empty lists, drop the column
                    if non_empty_cells_count == 0:
                        df.drop([column], axis=1, inplace=True)
                    # If no cell has more than one element, reduce from list to only the first element (or None if empty)
                    elif greater_than_one_cells_count == 0:
                        df[column] =  df[column].apply(lambda x: int(x[0]) if len(x)>0 else None).astype('Int64')
                        
    

    def transform(self, data_json:dict[str, dict] = None) -> dict[str, pd.DataFrame]:
        """
        Cleans the data from JSON format to Pandas' DataFrames. Uses preferencially the data passed as argument, 
        if it's None uses the data stored in self.data_json, passed when creating the class or when this method was last called.

        Args:
            data_json: Data to be transformed, in JSON format.

        Returns:
            dict[str, pd.DataFrame]: Generated DataFrames, identified by the name of the resource.
        """

        if(data_json != None):
            self.data_json = data_json

        self.__convert_to_raw_dataframes()
        self.__create_new_columns()
        self.__convert_datatypes()
        self.__identify_resources_by_id()
        self.__drop_irrelevant_columns()
        self.__filter_list_columns()

        return self.data_DFs
    
dt = DataTransformator(data_json)
data_DFs = dt.transform()

In [139]:
data_DFs["characters"].head(5)

,id,name,modified,comics,series,stories,events
0,1011334,3-D Man,2013-06-28 16:31:24-04:00,"[21366, 24571, 21546, 21741, 21975, 22299, 223...","[1945, 2005, 2045]","[19947, 19948, 19949, 19950, 19951, 19952, 543...",[269]
1,1017100,A-Bomb (HAS),2014-06-13 11:42:39-04:00,"[47176, 40632, 40630, 40628]","[17765, 3374]","[92086, 92087, 92082, 92083, 92078, 92079, 105...",[]
2,1009144,A.I.M.,2014-03-25 15:39:52-04:00,"[36763, 17553, 7340, 4214, 63217, 63218, 63219...","[13082, 1991, 354, 23123, 1046, 27689, 35600, ...","[10253, 10255, 10256, 10259, 10261, 10262, 119...",[]
3,1010699,Aaron Stack,2013-06-28 18:34:27-04:00,"[40776, 40773, 40774, 40778, 40787, 40786, 380...","[789, 3374, 2085]","[92376, 92377, 92370, 92371, 92372, 92373, 923...",[]
4,1009146,Abomination (Emil Blonsky),2015-10-01 17:37:36-04:00,"[88869, 88870, 88871, 17547, 17548, 1098, 8557...","[30582, 354, 158, 378, 1806, 3374, 465, 924, 2...","[4946, 5496, 12370, 12372, 18419, 18420, 18776...",[296]


In [140]:
data_DFs["comics"].head(5)

,id,title,issueNumber,modified,pageCount,series,creators,characters,stories,saleDate,focDate,printPrice,digitalPrice
0,82967,Marvel Previews (2017),0,2013-06-28 16:31:24-04:00,112,23665,[10021],[],"[183698, 183699]",2099-10-30 00:00:00-05:00,2019-10-07 00:00:00-04:00,0.00,NaN
1,82965,Marvel Previews (2017),0,2014-06-13 11:42:39-04:00,152,23665,[],[],[],2099-08-28 00:00:00-05:00,2019-08-05 00:00:00-04:00,0.00,NaN
2,82970,Marvel Previews (2017),0,2014-03-25 15:39:52-04:00,112,23665,[10021],[],[183704],2099-01-29 00:00:00-05:00,2020-01-06 00:00:00-05:00,0.00,0.0
3,15094,Silver Surfer (1987),0,2013-06-28 18:34:27-04:00,0,2288,[],[],"[30658, 67348]",2029-12-31 00:00:00-05:00,None,0.00,NaN
4,1886,Official Handbook of the Marvel Universe (2004...,12,2015-10-01 17:37:36-04:00,0,787,"[907, 887, 902, 909, 906, 910, 897, 900, 905, ...","[1009156, 1009197, 1009243, 1009313, 1009349, ...","[4430, 4431]",2029-12-31 00:00:00-05:00,None,3.99,NaN


In [141]:
data_DFs["creators"].head(5)

,id,firstName,middleName,lastName,suffix,fullName,modified,comics,series,stories,events
0,13970,#O,,,,#O,2013-06-28 16:31:24-04:00,"[8546, 10492, 15902, 11914]","[378, 2059, 2431, 2085]","[25655, 26016, 26318]",<NA>
1,13971,#X,,,,#X,2014-06-13 11:42:39-04:00,"[4241, 8547, 69274, 11901]","[378, 1806, 2059, 2085]","[24903, 24941, 25632, 152331]",<NA>
2,6606,A.R.K.,,,,A.R.K.,2014-03-25 15:39:52-04:00,[],[],[],<NA>
3,1168,All Thumbs Creative,,,,All Thumbs Creative,2013-06-28 18:34:27-04:00,"[68960, 28003, 25797, 62853, 60438, 66207, 218...","[25021, 8967, 8031, 23051, 22319, 24004, 5398,...","[47950, 53831, 56854, 61249, 61285, 75266, 909...",<NA>
4,7470,ALSJOERDSMA,,,,ALSJOERDSMA,2015-10-01 17:37:36-04:00,[],[],[],<NA>


In [142]:
data_DFs["events"].head(5)

,id,title,modified,start,end,creators,characters,stories,comics,series,next,previous
0,116,Acts of Vengeance!,2013-06-28 16:31:24-04:00,1989-12-10,2008-01-04,"[2707, 2077, 1277, 5823, 87, 211, 1216, 1827, ...","[1009148, 1009435, 1010370, 1009152, 1009156, ...","[21328, 21329, 21333, 21334, 16111, 16112, 148...","[12744, 12746, 7188, 7189, 7190, 6933, 17774, ...","[2116, 1991, 1988, 3462, 1996, 2002, 3741, 212...",240,233
1,227,Age of Apocalypse,2014-06-13 11:42:39-04:00,1995-03-01,1996-06-01,"[807, 3626, 937, 506, 1192, 16, 277, 11063, 71...","[1009149, 1010903, 1009153, 1009156, 1009158, ...","[6850, 37987, 37989, 37991, 37993, 38624, 3862...","[17701, 17731, 17732, 17733, 17734, 17735, 177...","[3614, 3618, 3619, 3628, 3737, 2578, 3734, 367...",239,219
2,314,Age of Ultron,2014-03-25 15:39:52-04:00,2013-03-06,2013-06-19,"[4994, 11748, 357, 12297, 820, 208, 24, 10023,...","[1009155, 1009165, 1009189, 1009220, 1009281, ...","[105614, 105615, 82809, 82810, 82811, 82812, 8...","[37405, 37406, 37407, 45904, 45905, 45906, 459...","[17318, 17370, 18248, 17554, 17900, 16414, 13910]",315,311
3,303,Age of X,2013-06-28 18:34:27-04:00,2011-01-26,2011-05-01,"[232, 648, 412, 4981, 5251, 366, 3736, 9967, 6...","[1009158, 1009164, 1009165, 1009175, 1009199, ...","[80892, 80893, 80904, 80905, 81019, 81020, 810...","[37996, 38524, 38523, 36495, 36502, 36497, 364...","[13603, 13896, 7455, 3754]",302,296
4,329,All-New All-Different Marvel,2015-10-01 17:37:36-04:00,NaT,NaT,[],[],[],[],[],None,None


In [143]:
data_DFs["series"].head(5)

,id,title,startYear,endYear,rating,type,modified,creators,characters,stories,comics,next,previous
0,31445,Fantastic Four by Dan Slott Vol. 1 (2021),2021,2021,,collection,2013-06-28 16:31:24-04:00,[4430],[],"[205070, 205071]",[91992],None,None
1,26024,Superior Spider-Man Vol. 2: Otto-matic (2019),2019,2019,,collection,2014-06-13 11:42:39-04:00,"[11765, 942, 437, 4430]",[1009610],"[158776, 158777]",[71400],None,None
2,18454,100th Anniversary Special (2014),2014,2014,Rated T,limited,2014-03-25 15:39:52-04:00,"[485, 9432, 11788, 13138, 12392]","[1009165, 1009299, 1009312, 1011299, 1009389, ...","[110101, 110102, 110103, 110104, 110105, 11010...","[49007, 49008, 49009, 49010, 49011]",None,None
3,13379,15 Love (2011),2011,2011,,,2013-06-28 18:34:27-04:00,"[274, 496, 8822]",[],"[82480, 82481, 82484, 82485, 82486, 82487, 106...","[37254, 37251, 37253]",None,None
4,13380,15-Love GN-TPB (2013 - Present),2013,2099,,,2015-10-01 17:37:36-04:00,[],[],"[82482, 82483]",[37252],None,None


In [144]:
data_DFs["stories"].head(5)

,id,title,modified,creators,series,comics
0,7,"Investigating the murder of a teenage girl, Ca...",2013-06-28 16:31:24-04:00,[],6,941
1,8,"In the wake of September 11th, the world watch...",2014-06-13 11:42:39-04:00,[],7,942
2,9,Ordinary New York City cop Frankie &QUOT;Gunz&...,2014-03-25 15:39:52-04:00,[],8,943
3,10,"In this thought-provoking anthology, a world-c...",2013-06-28 18:34:27-04:00,"[14458, 13567, 5187, 365, 4282, 9181, 441, 133...",9,944
4,11,Interior #11,2015-10-01 17:37:36-04:00,[],10,945


# Data analysis